# UMF Solver Examples

This notebook demonstrates the glaze solver - finding material combinations that match a target UMF.

Use cases:
- **Substitute materials**: "I don't have Custer Feldspar, what can I use instead?"
- **Match a published UMF**: "I want this UMF with my available materials"
- **Constrain materials**: "Use at least 20% ball clay for plasticity"

In [1]:
from solver import solve_umf_match, recipe_to_umf, format_solution
from utils import read_materials, read_recipes, read_umf, read_constraints, format_umf_table

## Material Library

Common ceramic materials with oxide analyses from Digitalfire and Glazy.org.

In [2]:
materials = read_materials("""
materials:
  # === FELDSPARS ===
  custer_feldspar:
    name: Custer Feldspar
    loi: 0.3
    analysis:
      SiO2: 68.50
      Al2O3: 17.00
      K2O: 10.00
      Na2O: 3.00
      CaO: 0.30

  g200_feldspar:
    name: G-200 Feldspar
    loi: 0.15
    analysis:
      SiO2: 66.30
      Al2O3: 18.50
      K2O: 10.80
      Na2O: 3.10
      CaO: 0.80

  minspar_200:
    name: Minspar 200
    loi: 0.30
    analysis:
      SiO2: 68.80
      Al2O3: 18.20
      Na2O: 6.50
      K2O: 4.10
      CaO: 1.50

  nepheline_syenite:
    name: Nepheline Syenite
    loi: 0.42
    analysis:
      SiO2: 60.20
      Al2O3: 23.60
      Na2O: 10.50
      K2O: 4.80
      CaO: 0.35

  # === CLAYS ===
  epk:
    name: EPK Kaolin
    loi: 13.20
    analysis:
      SiO2: 45.73
      Al2O3: 37.36
      K2O: 0.33
      CaO: 0.18

  tile_6:
    name: Tile #6 Kaolin
    loi: 13.50
    analysis:
      SiO2: 45.50
      Al2O3: 38.10
      TiO2: 1.40

  om4_ball_clay:
    name: OM4 Ball Clay
    loi: 9.30
    analysis:
      SiO2: 59.70
      Al2O3: 26.30
      K2O: 1.10
      Na2O: 0.20
      CaO: 0.20
      MgO: 0.50

  grolleg:
    name: Grolleg Kaolin
    loi: 12.01
    analysis:
      SiO2: 48.00
      Al2O3: 37.00
      K2O: 1.90
      CaO: 0.10

  # === CALCIUM SOURCES ===
  whiting:
    name: Whiting
    loi: 43.90
    analysis:
      CaO: 56.10

  wollastonite:
    name: Wollastonite
    analysis:
      CaO: 48.28
      SiO2: 51.72

  dolomite:
    name: Dolomite
    loi: 47.61
    analysis:
      CaO: 30.49
      MgO: 21.90

  # === SILICA ===
  silica:
    name: Silica
    analysis:
      SiO2: 100.00

  # === OTHER FLUXES ===
  talc:
    name: Talc
    loi: 4.75
    analysis:
      SiO2: 63.38
      MgO: 31.87

  zinc_oxide:
    name: Zinc Oxide
    analysis:
      ZnO: 100.00
""")

## Example 1: Match the Leach 4321

The Leach 4321 is a classic cone 10 clear glaze:
- 40 Feldspar, 30 Silica, 20 Whiting, 10 Kaolin

Let's see if the solver can find this recipe from just the target UMF.

In [3]:
leach = read_recipes("""
recipes:
  leach_4321:
    name: Leach 4321
    materials:
      custer_feldspar: 40
      silica: 30
      whiting: 20
      epk: 10
""")

leach_recipe = leach['leach_4321']['materials']
leach_umf = recipe_to_umf(leach_recipe, materials)
print(format_umf_table(leach_umf))

Flux:
  Na2O     0.073
  K2O      0.162
  CaO      0.765
  TOTAL    1.000

Other:
  SiO2     3.896
  Al2O3    0.390


In [4]:
# Use only the classic materials
classic_materials = {k: materials[k] for k in ['custer_feldspar', 'silica', 'whiting', 'epk']}

solution = solve_umf_match(leach_umf, classic_materials)
print(format_solution(solution, classic_materials))

/home/dauglyon/projects/glaze-chem/.venv/lib/python3.13/site-packages/scipy/optimize/_differentiable_functions.py:737: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


Recipe:
------------------------------
  Custer Feldspar        40.0
  Silica                 30.0
  Whiting                20.0
  EPK Kaolin             10.0
  TOTAL                 100.0

Resulting UMF:
------------------------------
Flux:
  Na2O     0.073
  K2O      0.162
  CaO      0.765
  TOTAL    1.000

Other:
  SiO2     3.896
  Al2O3    0.390


## Example 2: Feldspar Substitution

"I have a recipe calling for Custer Feldspar, but I only have Minspar 200. What adjustments do I need?"

First, compute the original recipe's UMF, then solve with different materials.

In [5]:
original = read_recipes("""
recipes:
  my_clear:
    name: My Clear Glaze
    materials:
      custer_feldspar: 45
      silica: 25
      whiting: 20
      epk: 10
""")

original_recipe = original['my_clear']['materials']
original_umf = recipe_to_umf(original_recipe, materials)
print(format_umf_table(original_umf))

Flux:
  Na2O     0.080
  K2O      0.176
  CaO      0.744
  TOTAL    1.000

Other:
  SiO2     3.686
  Al2O3    0.409


In [6]:
# Solve using Minspar instead of Custer
available = {k: materials[k] for k in ['minspar_200', 'silica', 'whiting', 'epk']}

solution = solve_umf_match(original_umf, available)
print(format_solution(solution, available))

Recipe:
------------------------------
  Minspar 200            42.4
  Silica                 27.3
  Whiting                20.1
  EPK Kaolin             10.3
  TOTAL                 100.0

Resulting UMF:
------------------------------
Flux:
  Na2O     0.161
  K2O      0.068
  CaO      0.771
  TOTAL    1.000

Other:
  SiO2     3.686
  Al2O3    0.411

Error (result - target):
------------------------------
  Al2O3    +0.001
  CaO      +0.027
  K2O      -0.108
  Na2O     +0.081


## Example 3: Constrained Solving

"I want at least 15% ball clay for plasticity, and I'd like to use wollastonite instead of whiting."

In [7]:
target_umf = read_umf("""
name: Cone 6 Clear
flux:
  K2O: 0.20
  Na2O: 0.05
  CaO: 0.75
other:
  Al2O3: 0.40
  SiO2: 3.80
""")

available = read_materials("""
materials:
  custer_feldspar:
    name: Custer Feldspar
    analysis: {SiO2: 68.5, Al2O3: 17.0, K2O: 10.0, Na2O: 3.0}
  silica:
    name: Silica
    analysis: {SiO2: 100.0}
  wollastonite:
    name: Wollastonite
    analysis: {CaO: 48.28, SiO2: 51.72}
  om4_ball_clay:
    name: OM4 Ball Clay
    analysis: {SiO2: 59.7, Al2O3: 26.3, K2O: 1.1, Na2O: 0.2, CaO: 0.2, MgO: 0.5}
  epk:
    name: EPK Kaolin
    analysis: {SiO2: 45.73, Al2O3: 37.36}
""")

constraints = read_constraints("""
om4_ball_clay:
  min: 15
""")

solution = solve_umf_match(target_umf, available, constraints)
print(format_solution(solution, available))

Recipe:
------------------------------
  Custer Feldspar        47.2
  Wollastonite           25.3
  OM4 Ball Clay          15.0
  Silica                 12.6
  TOTAL                 100.0

Resulting UMF:
------------------------------
Flux:
  Na2O     0.079
  K2O      0.176
  CaO      0.739
  MgO      0.006
  TOTAL    1.000

Other:
  SiO2     3.776
  Al2O3    0.398

Error (result - target):
------------------------------
  Al2O3    -0.002
  CaO      -0.011
  K2O      -0.024
  Na2O     +0.029
  SiO2     -0.024


/home/dauglyon/projects/glaze-chem/.venv/lib/python3.13/site-packages/scipy/optimize/_differentiable_functions.py:385: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


## Example 4: Fixed Material Amounts

"I want exactly 25% wollastonite and 10% talc for a magnesia-calcia glaze. Find the rest."

In [8]:
target_umf = read_umf("""
name: Magnesia-Calcia Matte
flux:
  K2O: 0.15
  CaO: 0.60
  MgO: 0.25
other:
  Al2O3: 0.35
  SiO2: 3.50
""")

available = read_materials("""
materials:
  custer_feldspar:
    name: Custer Feldspar
    analysis: {SiO2: 68.5, Al2O3: 17.0, K2O: 10.0, Na2O: 3.0}
  nepheline_syenite:
    name: Nepheline Syenite
    analysis: {SiO2: 60.2, Al2O3: 23.6, Na2O: 10.5, K2O: 4.8}
  silica:
    name: Silica
    analysis: {SiO2: 100.0}
  wollastonite:
    name: Wollastonite
    analysis: {CaO: 48.28, SiO2: 51.72}
  talc:
    name: Talc
    analysis: {SiO2: 63.38, MgO: 31.87}
  epk:
    name: EPK Kaolin
    analysis: {SiO2: 45.73, Al2O3: 37.36}
""")

constraints = read_constraints("""
wollastonite:
  fixed: 25
talc:
  fixed: 10
""")

solution = solve_umf_match(target_umf, available, constraints)
print(format_solution(solution, available))

Recipe:
------------------------------
  Custer Feldspar        27.7
  Wollastonite           25.0
  Silica                 22.9
  EPK Kaolin             14.4
  Talc                   10.0
  TOTAL                 100.0

Resulting UMF:
------------------------------
Flux:
  Na2O     0.040
  K2O      0.087
  CaO      0.638
  MgO      0.235
  TOTAL    1.000

Other:
  SiO2     3.343
  Al2O3    0.293

Error (result - target):
------------------------------
  Al2O3    -0.057
  CaO      +0.038
  K2O      -0.063
  MgO      -0.015
  SiO2     -0.157


## Example 5: Multiple Feldspar Options

Give the solver several feldspars and let it pick the best combination.

In [9]:
target_umf = read_umf("""
name: Balanced Alkali Clear
flux:
  K2O: 0.15
  Na2O: 0.15
  CaO: 0.70
other:
  Al2O3: 0.45
  SiO2: 4.00
""")

available = read_materials("""
materials:
  custer_feldspar:
    name: Custer Feldspar
    analysis: {SiO2: 68.5, Al2O3: 17.0, K2O: 10.0, Na2O: 3.0}
  minspar_200:
    name: Minspar 200
    analysis: {SiO2: 68.8, Al2O3: 18.2, Na2O: 6.5, K2O: 4.1, CaO: 1.5}
  nepheline_syenite:
    name: Nepheline Syenite
    analysis: {SiO2: 60.2, Al2O3: 23.6, Na2O: 10.5, K2O: 4.8}
  silica:
    name: Silica
    analysis: {SiO2: 100.0}
  whiting:
    name: Whiting
    analysis: {CaO: 56.1}
  epk:
    name: EPK Kaolin
    analysis: {SiO2: 45.73, Al2O3: 37.36}
""")

solution = solve_umf_match(target_umf, available)
print(format_solution(solution, available))

Recipe:
------------------------------
  Custer Feldspar        26.5
  Minspar 200            24.0
  Silica                 23.7
  Whiting                17.5
  EPK Kaolin              7.7
  Nepheline Syenite       0.5
  TOTAL                 100.0

Resulting UMF:
------------------------------
Flux:
  Na2O     0.150
  K2O      0.150
  CaO      0.700
  TOTAL    1.000

Other:
  SiO2     4.000
  Al2O3    0.450


## Solver Limitations

The solver finds the best *chemical* match, but can't account for:

- **Working properties**: Ball clay vs kaolin affects plasticity
- **Melting behavior**: Same UMF can melt differently depending on materials
- **Particle size**: Affects melt, suspension, application
- **Surface texture**: Wollastonite vs whiting can affect surface

Always test substitutions before committing to a batch.